## Script is designed to create the taxonomy table file

Output is: <br>

1. <i> File "Tax_gtdb.csv" <br></i>
Taxonomy table file. Taxonomy was taken from last GTDB release (release207/207.0) <br>
https://data.ace.uq.edu.au/public/gtdb/data/releases/release207/207.0/ <br>
<br>
2. <i> File "Tax_gtdb-ncbi.csv" <br></i>
Taxonomy table file. The table includes the gtdb taxonomy and ncbi names for all species for which I find a correspondence between dtdb and ncbi spesies names. 
Gtdb species names vs ncbi species names match files were taken from here: <br>
https://data.gtdb.ecogenomic.org/releases/latest/auxillary_files/


In [2]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd

## Tax_gtdb

In [40]:
path='/Users/katerynapantiukh/Documents/1MyDisk/Code/Microbiome/'
colnames = ['dom','phylum','class','order','family','genus','sp']

# release 207
#bac = pd.read_csv('input/bac120_taxonomy_r207.tsv', sep=';', names=colnames, header=None)
#arh = pd.read_csv('input/ar53_taxonomy_r207.tsv', sep=';', names=colnames, header=None)

# release 214
bac = pd.read_csv(str(path)+'/gtdb_vizualization/input/bac120_taxonomy_r214.tsv', sep=';', names=colnames, header=None)
arh = pd.read_csv(str(path)+'gtdb_vizualization/input/ar53_taxonomy_r214.tsv', sep=';', names=colnames, header=None)

# release 220
#bac = pd.read_csv(str(path)+'/gtdb_vizualization/input/bac120_taxonomy_r220.tsv', sep=';', names=colnames, header=None)
#arh = pd.read_csv(str(path)+'gtdb_vizualization/input/ar53_taxonomy_r220.tsv', sep=';', names=colnames, header=None)

# modify bac120 table
bac['domain'] = bac['dom'].str.split('__', expand=True)[1]
bac['phylum'] = bac['phylum'].str.replace(r'p__', '')
bac['class'] = bac['class'].str.replace(r'c__', '')
bac['order'] = bac['order'].str.replace(r'o__', '')
bac['family'] = bac['family'].str.replace(r'f__', '')
bac['genus'] = bac['genus'].str.replace(r'g__', '')
bac['sp'] = bac['sp'].str.replace(r's__', '')
bac = bac[['domain','phylum','class','order','family','genus','sp']]
bac = bac.drop_duplicates()

# modify arch table
arh['domain'] = arh['dom'].str.split('__', expand=True)[1]
arh['phylum'] = arh['phylum'].str.replace(r'p__', '')
arh['class'] = arh['class'].str.replace(r'c__', '')
arh['order'] = arh['order'].str.replace(r'o__', '')
arh['family'] = arh['family'].str.replace(r'f__', '')
arh['genus'] = arh['genus'].str.replace(r'g__', '')
arh['sp'] = arh['sp'].str.replace(r's__', '')
arh = arh[['domain','phylum','class','order','family','genus','sp']]
arh = arh.drop_duplicates()

frames = [bac,arh]
gtdb = pd.concat(frames)

gtdb.to_excel('results/GTDB_all_species.xlsx', index=False)
gtdb.head(2)

,domain,phylum,class,order,family,genus,sp
0,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli
33849,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae


In [32]:
print('Total bacteria species number in GTDB is ',len(gtdb.loc[gtdb['domain'] == 'Bacteria']))

Total bacteria species number in GTDB is  107235


## Tax_gtdb-ncbi

In [33]:
# for release 214!!! Change files for the new releases
rel = '220'
convB = pd.read_excel('input/gtdb_vs_ncbi_r'+str(rel)+'_bacteria.xlsx', sheet_name='Species')
print('Unique bacteria species by gtdb - ',len(convB['GTDB R220 species'].unique()))

convB.rename(columns = {'GTDB R220 species':'sp_gtdb',"List of NCBI species":'ncbi'}, inplace = True)
new = convB.ncbi.str.rsplit(' ', 1, expand=True)\
  .rename(columns=lambda x: 'col{}'.format(x + 1))
convB['sp_ncbi'] = new['col1']
convB['sp_ncbi'] = convB['sp_ncbi'].str.replace(r's__', '')
convB['sp_gtdb'] = convB['sp_gtdb'].str.replace(r's__', '')
convB = convB[['sp_gtdb','sp_ncbi']]
convB = convB.loc[convB['sp_ncbi'] != "(g__)"]
convB = convB.reset_index(drop=True)

convB.tail(2)

Unique bacteria species by gtdb -  97786


/var/folders/5j/pvj6k02n3mn9ztpn77vjtfzw0000gn/T/ipykernel_92225/4009286634.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  new = convB.ncbi.str.rsplit(' ', 1, expand=True)\


,sp_gtdb,sp_ncbi
14238,Streptococcus parasanguinis_H,Streptococcus parasanguinis
14239,Paenibacillus_E whitsoniae,Paenibacillus whitsoniae


In [35]:
convA = pd.read_excel('input/gtdb_vs_ncbi_r'+str(rel)+'_archaea.xlsx', sheet_name='Species')
print('Unique archaea species by gtdb - ',len(convA['GTDB R220 species'].unique()))

convA['top_sp_ncbi'] = convA['List of NCBI species'].str.split(',', expand=True)[0]
convA['GTDB R220 species'] = convA['GTDB R220 species'].str.replace(r's__', '')
convA['top_sp_ncbi'] = convA['top_sp_ncbi'].str.replace(r's__', '')
convA.rename(columns = {'GTDB R220 species':'sp_gtdb'}, inplace = True)
new = convA.top_sp_ncbi.str.rsplit(' ', 1, expand=True)\
  .rename(columns=lambda x: 'col{}'.format(x + 1))
convA['sp_ncbi'] = new['col1']

convA = convA[['sp_gtdb','sp_ncbi']]
convA = convA.loc[convA['sp_ncbi'] != "(g__)"]
convA = convA.reset_index(drop=True)

convA.tail(2)

Unique archaea species by gtdb -  5455


/var/folders/5j/pvj6k02n3mn9ztpn77vjtfzw0000gn/T/ipykernel_92225/1830643434.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  new = convA.top_sp_ncbi.str.rsplit(' ', 1, expand=True)\


,sp_gtdb,sp_ncbi
5453,JABDBI01 sp013288705,(g__Methanobrevibacter)
5454,JABEUN01 sp021836665,(g__Methanobrevibacter)


In [36]:
frames = [convA,convB]
conv = pd.concat(frames)
conv = conv.reset_index(drop=True)

print('Unique species by gtdb - ',len(conv['sp_gtdb'].unique()))
conv.tail(2)

Unique species by gtdb -  19692


,sp_gtdb,sp_ncbi
19693,Streptococcus parasanguinis_H,Streptococcus parasanguinis
19694,Paenibacillus_E whitsoniae,Paenibacillus whitsoniae


In [37]:
gtdbN = pd.merge(gtdb, conv, how="inner", left_on="sp", right_on='sp_gtdb').sort_values('sp_ncbi').reset_index(drop=True)
gtdbN.to_excel('results/Tax_gtdb-ncbi_v'+str(rel)+'.xlsx', index=False)

gtdbN.tail(2)

,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi
19693,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Mediterraneibacter,Mediterraneibacter lactaris,Mediterraneibacter lactaris,"[Ruminococcus] lactaris 81.97%, (g__)"
19694,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Mediterraneibacter,Mediterraneibacter torques,Mediterraneibacter torques,"[Ruminococcus] torques 72.94%, (g__) 23.53%, F..."


### Random check merge table

In [38]:
gtdbN.loc[gtdbN['sp_gtdb'] == 'Klebsiella pneumoniae']

,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi
14026,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae,Klebsiella pneumoniae,"Klebsiella pneumoniae 99.63%, (g__) 0.32%, Esc..."


## Check stats

In [30]:
print('For db version',rel)
print('Total GTDB bacteria species in initial table is:',len(gtdb.loc[gtdb['domain'] == 'Bacteria']))
print('Total bacteria species in GTDB-NCBI result table is ',len(gtdbN.loc[gtdbN['domain'] == 'Bacteria']))

For db version 214
Total GTDB bacteria species in initial table is: 80789
Total bacteria species in GTDB-NCBI result table is  11972


In [39]:
print('For db version',rel)
print('Total GTDB bacteria species in initial table is:',len(gtdb.loc[gtdb['domain'] == 'Bacteria']))
print('Total bacteria species in GTDB-NCBI result table is ',len(gtdbN.loc[gtdbN['domain'] == 'Bacteria']))

For db version 220
Total GTDB bacteria species in initial table is: 107235
Total bacteria species in GTDB-NCBI result table is  14258
